# 1 Data preprocessing

## 1.1 Importing modules and reading data

In [1]:
import os
import pandas as pd
from sklearn.preprocessing import StandardScaler
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB

In [2]:
filepath = '/kaggle/input/icr-identify-age-related-conditions/'
data = pd.read_csv(os.path.join(filepath, 'train.csv'))
print(data.shape)

data = data.drop('Id', axis=1)
data.head()

(617, 58)


,AB,AF,AH,AM,AR,AX,AY,AZ,BC,BD,...,FL,FR,FS,GB,GE,GF,GH,GI,GL,Class
0,0.209377,3109.03329,85.200147,22.394407,8.138688,0.699861,0.025578,9.812214,5.555634,4126.58731,...,7.298162,1.73855,0.094822,11.339138,72.611063,2003.810319,22.136229,69.834944,0.120343,1
1,0.145282,978.76416,85.200147,36.968889,8.138688,3.632190,0.025578,13.517790,1.229900,5496.92824,...,0.173229,0.49706,0.568932,9.292698,72.611063,27981.562750,29.135430,32.131996,21.978000,0
2,0.470030,2635.10654,85.200147,32.360553,8.138688,6.732840,0.025578,12.824570,1.229900,5135.78024,...,7.709560,0.97556,1.198821,37.077772,88.609437,13676.957810,28.022851,35.192676,0.196941,0
3,0.252107,3819.65177,120.201618,77.112203,8.138688,3.685344,0.025578,11.053708,1.229900,4169.67738,...,6.122162,0.49706,0.284466,18.529584,82.416803,2094.262452,39.948656,90.493248,0.155829,0
4,0.380297,3733.04844,85.200147,14.103738,8.138688,3.942255,0.054810,3.396778,102.151980,5728.73412,...,8.153058,48.50134,0.121914,16.408728,146.109943,8524.370502,45.381316,36.262628,0.096614,1


## 1.2 Data cleaning

### Checking for duplicates

In [3]:
duplicates = data.duplicated()
print("Number of duplicates:", duplicates.sum())

Number of duplicates: 0


### Categorical encoding

In [4]:
# Create a sample DataFrame
data['EJ'] = data['EJ'].replace('A', 0)
data['EJ'] = data['EJ'].replace('B', 1)

# Convert 'Column1' from object to integer
data['EJ'] = data['EJ'].astype(int)

### Dealing with insignificant missing data

In [5]:
missing_values = data.isna().sum().sum()
print("Number of missing values:", missing_values)

print("Shape of the data before dropping NULL values:", data.shape)

missing_names = []
for column_name, column_data in data.items():
    # Perform operations on each column
    missing_percentage = (column_data.isnull().sum() / len(column_data)) * 100
    if missing_percentage < 9:
        data.dropna(subset=[column_name], inplace=True)
    else:
        missing_names.append(column_name) 
        

print("Shape of the data before after NULL values:", data.shape)

Number of missing values: 131
Shape of the data before dropping NULL values: (617, 57)
Shape of the data before after NULL values: (612, 57)


### Separating responses from predictors before regressing data

In [6]:
y = data['Class']  # Extract the response variable column(s)
X = data.copy()  # Remove the response variable column(s) from the DataFrame

print(missing_names)

['BQ', 'EL']


### Regressing missing data

In [7]:
#Regressing missing BQ data

train_X_BQ = X.drop('EL', axis=1)
full_BQ = train_X_BQ.copy()
test_X_BQ = train_X_BQ[train_X_BQ['BQ'].isnull()].copy()
test_X_BQ = test_X_BQ.drop('BQ', axis=1)

train_X_BQ.dropna(inplace=True)

train_Y_BQ = train_X_BQ['BQ']
train_X_BQ = train_X_BQ.drop('BQ', axis=1)

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

regressor = LinearRegression()
regressor.fit(train_X_BQ, train_Y_BQ)
test_Y_BQ = regressor.predict(test_X_BQ)

full_BQ.loc[full_BQ['BQ'].isnull(), 'BQ'] = test_Y_BQ
X['BQ'].fillna(full_BQ['BQ'], inplace=True)

In [8]:
#Regressing missing EL data

train_X_EL = X.copy()
full_EL = train_X_EL.copy()
test_X_EL = train_X_EL[train_X_EL['EL'].isnull()].copy()
test_X_EL = test_X_EL.drop('EL', axis=1)

train_X_EL.dropna(inplace=True)

train_Y_EL = train_X_EL['EL']
train_X_EL = train_X_EL.drop('EL', axis=1)

regressor_EL = LinearRegression()
regressor_EL.fit(train_X_EL, train_Y_EL)
test_Y_EL = regressor_EL.predict(test_X_EL)

full_EL.loc[full_EL['EL'].isnull(), 'EL'] = test_Y_EL
X['EL'].fillna(full_EL['EL'], inplace=True)

In [9]:
X.columns.tolist()
correlation_matrix = X.corr()

class_correlations = correlation_matrix['Class']

low_correlation_columns = class_correlations[abs(class_correlations) < 0.05]

high_correlation_columns = class_correlations[abs(class_correlations) > 0.8]

print("Columns with low correlation (absolute value < 0.05):")
for column, correlation in low_correlation_columns.items():
    print(f"{column}: {abs(correlation)}")

print("\nColumns with high correlation (absolute value > 0.8):")
for column, correlation in high_correlation_columns.items():
    print(f"{column}: {abs(correlation)}")

Columns with low correlation (absolute value < 0.05):
AH: 0.04209024672535381
AZ: 0.01270428917437259
CB: 0.025776691867449207
CH: 0.0033435657242242853
CL: 0.014922630449296549
CS: 0.04964172746379496
DN: 0.027359905615540175
DV: 0.012521335596920034
EG: 0.0343881237284732
EU: 0.040800103477387994
FC: 0.030450532282503075
FS: 0.0011210074041133038
GH: 0.027651325525788365

Columns with high correlation (absolute value > 0.8):
Class: 1.0


In [10]:
col_drops = ['AH', 'AZ','CB', 'CH','CL','CS','DN','DV','EG','EU','FC','FS','GH'] 
X.drop(col_drops,axis=1,inplace=True) 

### Normalizing and Splitting data

In [11]:
X = X.drop('Class', axis=1)
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [12]:
from sklearn.model_selection import train_test_split 
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0) 

## 1.3 Exploring data   

# 2 Training models

## 2.1 Training models

In [13]:
logreg = LogisticRegression()

# Fit the model to the training data
logreg.fit(X_train, y_train)

#logistic_regression.fit(X_train, y_train)

# Predict the target variable for the test set
y_pred = logreg.predict(X_test)

# Calculate accuracy
accuracy_lr = accuracy_score(y_test, y_pred)
print("Accuracy of Logistic Regression:", accuracy_lr)

# Print confusion matrix
confusion_matrix_lr = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(confusion_matrix_lr)


random_forest_model = RandomForestClassifier()

# Train the model on the training data
random_forest_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred_rf = random_forest_model.predict(X_test)

# Calculate accuracy
accuracy_rf = accuracy_score(y_test, y_pred_rf)
print("Accuracy of Random Forest Clasifier:", accuracy_rf)

# Print confusion matrix
confusion_matrix_rf = confusion_matrix(y_test, y_pred_rf)
print("Confusion Matrix:")
print(confusion_matrix_rf)

# Initialize the SVC classifier
svc_model = SVC(kernel='rbf')
# Train the SVC model on the training data
svc_model.fit(X_train, y_train)

# Initialize the GaussianNB classifier
gnb_model = GaussianNB()
# Train the GaussianNB model on the training data
gnb_model.fit(X_train, y_train)

# Initialize the XGBClassifier
xgb_model = XGBClassifier()
# Train the XGBClassifier on the training data
xgb_model.fit(X_train, y_train)

# Make predictions on the test data
y_pred_svc = svc_model.predict(X_test)
y_pred_gnb = gnb_model.predict(X_test)
y_pred_xgb = xgb_model.predict(X_test)


# Calculate accuracy
accuracy_svc = accuracy_score(y_test, y_pred_svc)
print("Accuracy of Support Vector Machine:", accuracy_svc)


confusion_matrix_svc = confusion_matrix(y_test, y_pred_svc)
print("Confusion Matrix:")
print(confusion_matrix_svc)

# Calculate accuracy
accuracy_gnb = accuracy_score(y_test, y_pred_gnb)
print("Accuracy of Naive Bayes:", accuracy_gnb)


confusion_matrix_gnb = confusion_matrix(y_test, y_pred_gnb)
print("Confusion Matrix:")
print(confusion_matrix_gnb)

# Calculate accuracy
accuracy_xgb = accuracy_score(y_test, y_pred_xgb)
print("Accuracy of XGBoost:", accuracy_xgb)


confusion_matrix_xgb = confusion_matrix(y_test, y_pred_xgb)
print("Confusion Matrix:")
print(confusion_matrix_xgb)

Accuracy of Logistic Regression: 0.9512195121951219
Confusion Matrix:
[[103   2]
 [  4  14]]
Accuracy of Random Forest Clasifier: 0.959349593495935
Confusion Matrix:
[[104   1]
 [  4  14]]
Accuracy of Support Vector Machine: 0.926829268292683
Confusion Matrix:
[[105   0]
 [  9   9]]
Accuracy of Naive Bayes: 0.8861788617886179
Confusion Matrix:
[[102   3]
 [ 11   7]]
Accuracy of XGBoost: 0.959349593495935
Confusion Matrix:
[[102   3]
 [  2  16]]


## Fine-tuning top 3

In [14]:
# Initialize the Random Forest classifier
random_forest_best = RandomForestClassifier()

# Define the hyperparameters and their possible values
param_grid_rf_best = {
    'n_estimators': [100, 200, 300],  # Number of trees in the forest
    'max_depth': [None, 5, 10],       # Maximum depth of the trees
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split an internal node
}

# Perform grid search using 5-fold cross-validation
grid_search_rf_best = GridSearchCV(random_forest_best, param_grid_rf_best, cv=5)

# Fit the grid search to the training data
grid_search_rf_best.fit(X_train, y_train)

# Get the best parameters and the best model
best_params_rf_best = grid_search_rf_best.best_params_
best_model_rf_best = grid_search_rf_best.best_estimator_

# Make predictions on the test data using the best model
y_pred_rf_best = best_model_rf_best.predict(X_test)

accuracy_rf_best = accuracy_score(y_test, y_pred_rf_best)
print("Accuracy of best Random Forest:", accuracy_rf_best)

confusion_matrix_rf_best = confusion_matrix(y_test, y_pred_rf_best)
print("Confusion Matrix:")
print(confusion_matrix_rf_best)

Accuracy of best Random Forest: 0.975609756097561
Confusion Matrix:
[[105   0]
 [  3  15]]


In [15]:
# Initialize the XGBoost classifier
xgb_model_best = XGBClassifier()

# Define the hyperparameters and their possible values
param_grid_xgb_best = {
    'n_estimators': [100, 200, 300],  # Number of trees in the ensemble
    'max_depth': [3, 5, 7],           # Maximum depth of each tree
    'learning_rate': [0.1, 0.01, 0.001],  # Learning rate
}

# Perform grid search using 5-fold cross-validation
grid_search_xgb_best = GridSearchCV(xgb_model_best, param_grid_xgb_best, cv=5)

# Fit the grid search to the training data
grid_search_xgb_best.fit(X_train, y_train)

# Get the best parameters and the best model
best_params_xgb = grid_search_xgb_best.best_params_
best_model_xgb = grid_search_xgb_best.best_estimator_

# Make predictions on the test data using the best model
y_pred_xgb_best = best_model_xgb.predict(X_test)

accuracy_xgb_best = accuracy_score(y_test, y_pred_xgb_best)
print("Accuracy of best XGBoost:", accuracy_xgb_best)

confusion_matrix_xgb_best = confusion_matrix(y_test, y_pred_xgb_best)
print("Confusion Matrix:")
print(confusion_matrix_xgb_best)

Accuracy of best XGBoost: 0.959349593495935
Confusion Matrix:
[[102   3]
 [  2  16]]


In [16]:
# Initialize the Logistic Regression classifier
logreg_model_best = LogisticRegression()

# Define the hyperparameters and their possible values
param_grid_lr_best = {
    'C': [0.1, 1.0, 10.0],      # Inverse of regularization strength
    'penalty': ['l1', 'l2'],    # Regularization penalty type
    'solver': ['liblinear'],    # Algorithm to use in the optimization problem
}

# Perform grid search using 5-fold cross-validation
grid_search_lr_best = GridSearchCV(logreg_model_best, param_grid_lr_best, cv=5)

# Fit the grid search to the training data
grid_search_lr_best.fit(X_train, y_train)

# Get the best parameters and the best model
best_params_lr = grid_search_lr_best.best_params_
best_model_lr = grid_search_lr_best.best_estimator_

# Make predictions on the test data using the best model
y_pred_lr_best = best_model_lr.predict(X_test)

accuracy_lr_best = accuracy_score(y_test, y_pred_lr_best)
print("Accuracy of best Logistic Regression:", accuracy_lr_best)

confusion_matrix_lr_best = confusion_matrix(y_test, y_pred_lr_best)
print("Confusion Matrix:")
print(confusion_matrix_lr_best)

Accuracy of best Logistic Regression: 0.9512195121951219
Confusion Matrix:
[[103   2]
 [  4  14]]


# 3 Making a Submission

In [17]:
test_data = pd.read_csv('/kaggle/input/icr-identify-age-related-conditions/test.csv')

Id = test_data['Id'].values

col_drops = ['Id','AH', 'AZ','CB', 'CH','CL','CS','DN','DV','EG','EU','FC','FS','GH'] 
test_data.drop(col_drops,axis=1,inplace=True) 

test_data['EJ'] = test_data['EJ'].replace('A', 0)
test_data['EJ'] = test_data['EJ'].replace('B', 1)

# Convert 'Column1' from object to integer
test_data['EJ'] = test_data['EJ'].astype(int)

X_new = test_data.iloc[:, :].values 
y_new = best_model_xgb.predict(X_new)
print(y_new)

merge_data = {'Id': Id, 'class_0': y_new }

sub_data = pd.DataFrame(merge_data)
sub_data['class_1'] = 1 - sub_data['class_0']
sub_data.to_csv('submission.csv', index=False)

[0 0 0 0 0]
